In [ ]:
import numpy as np
import pandas as pd

In [ ]:
folder_path ="/Users/raphaelkronberg/Downloads/export_939_20220910_0950/__DUMMY_ID__820__/"
meta_data_file_path ="ecotaxa___DUMMY_ID__820__.tsv"
fpath = folder_path + meta_data_file_path
df_eco_taxa_820 = pd.read_csv(fpath, sep='\t')
df_eco_taxa_820.head()

In [ ]:
labels = df_eco_taxa_820[["img_file_name","object_annotation_category","object_annotation_category_id"]]
labels.shape

In [ ]:
labels.describe()

In [ ]:
labels

In [ ]:
label_dict = labels[["img_file_name","object_annotation_category_id"]]
id_name_map =labels[["object_annotation_category","object_annotation_category_id"]]

In [ ]:
label_dict = {row["img_file_name"]: row["object_annotation_category_id"] for index, row in labels.iterrows()}
len(label_dict)

In [ ]:
id_name_map_dict = {row["object_annotation_category"]: row["object_annotation_category_id"] for index, row in id_name_map.iterrows()}
len(id_name_map_dict)

In [ ]:
import os

In [ ]:
def create_label_image_mapping(path):
    df = pd.read_csv(path, sep='\t')[["img_file_name","object_annotation_category_id"]]
    label_dict = {row["img_file_name"]: row["object_annotation_category_id"] for index, row in df.iterrows()}
    print(len(label_dict))
    return label_dict, len(label_dict)

def create_label_id_label_name_mapping(path):
    df = pd.read_csv(path, sep='\t')[["object_annotation_category","object_annotation_category_id"]]
    id_name_map_dict = {row["object_annotation_category"]: row["object_annotation_category_id"] for index, row in df.iterrows()}
    print(len(id_name_map_dict))
    return id_name_map_dict, len(id_name_map_dict)


In [ ]:
label_img_map = {}
id_name_map ={}
counter = 0
for root, dirs, files in os.walk("/Users/raphaelkronberg/Downloads/export_939_20220910_0950"):
    for name in dirs:
        print(os.path.join(root, name))
    for name in files:
        if name.endswith('.tsv'):
            path = os.path.join(root, name)
            print(path)
            label_img_map[name], temp_len1 = create_label_image_mapping(path)
            id_name_map[name], temp_len2 = create_label_id_label_name_mapping(path)
            counter += temp_len1
            print('#'*20)
print('Number of Datapoints:', counter)

In [ ]:
len(label_img_map)

In [ ]:
list_of_ids = []
for key, val in label_img_map.items():
    for key1, val1 in val.items():
        list_of_ids.append(key1)
print(len(list_of_ids))
print(len(set(list_of_ids)))

In [ ]:
import collections
print([item for item, count in collections.Counter(list_of_ids).items() if count > 1])

In [ ]:
data_table = pd.DataFrame()
imgs = list()
labels = list()
for key, val in label_img_map.items():
    for key1, val1 in val.items():
        if key1 != "[t]":
            imgs.append(key1)
            labels.append(val1)
        else:
            print("[t]")
data_table['image'] = pd.Series(imgs, dtype='str')
data_table['labels'] = pd.Series(labels, dtype='str')
print(data_table.shape)
data_table.head(5)

In [ ]:
data_table.describe()

In [ ]:
different_labels = np.unique(labels)
print(different_labels)

In [ ]:
data_table.sort_values(by=['labels'],inplace = True)

In [ ]:
data_table.describe()

In [ ]:
id_over_treshhold = list()
counter_over_treshold = list()
for id in different_labels:
    if len(data_table[data_table["labels"]==id]) > 300:
        #print(f'id: {id}  count: {len(data_table[data_table["labels"]==id])}')
        id_over_treshhold.append(id)
        counter_over_treshold.append(len(data_table[data_table["labels"]==id]))
tresh_hold_df = pd.DataFrame()
tresh_hold_df['id'] =id_over_treshhold
tresh_hold_df['count'] = counter_over_treshold
tresh_hold_df.sort_values(by=['count'],inplace = True)


In [ ]:
data_map = pd.DataFrame()
id_lab = list()
name_lab = list()
for key, val in id_name_map.items():
    for key1, val1 in val.items():
        if key1 != "[t]":
            name_lab.append(key1)
            id_lab.append(val1)
        else:
            print("[t]")
data_map['id'] = pd.Series(id_lab, dtype='str')
data_map['name'] = pd.Series(name_lab, dtype='str')
print(data_map.shape)
data_map.drop_duplicates(inplace = True)
data_map
#len(data_map)

In [ ]:
data_map.keys()

In [ ]:
data_map_dict = {row["id"]: row["name"] for index, row in data_map.iterrows()}
def applydict(key):
    try:
        ret = data_map_dict[key]
        #print(type(key))
    except Exception as err:
        #print(type(key))
        print(err)
        ret = 'noname found'
    return ret
data_map_dict

In [ ]:
tresh_hold_df['label_name']= tresh_hold_df['id'].astype('str').apply(applydict)
len(tresh_hold_df)


In [ ]:
tresh_hold_df

In [ ]:
tresh_hold_df.to_csv('classes.csv', sep=';')

In [ ]:


def create_folder(path):
    try:
        os.makedirs(path)
    except FileExistsError:
        print("directory already exists")
        pass
    return None

create_folder("/Users/raphaelkronberg/PycharmProjects/deeploki/data/loki")

In [ ]:
folders_to_create = ['train', 'val', 'test']

In [ ]:
classes = list(tresh_hold_df['id'])
classes

In [ ]:
def is_in_df(key):
    if str(key) in list(tresh_hold_df['id']):
        ret = True
    else:
        ret = False
    return ret
print(list(tresh_hold_df['id']))

In [ ]:
from sklearn.model_selection import train_test_split
df = data_table
print(df.shape)
print(df.columns)
df['select'] = df['labels'].apply(is_in_df)
df = df[df['select']]
print(df.shape)
train, temp = train_test_split(df, test_size=0.2, train_size=0.8, random_state=42, shuffle=True, stratify=df['labels'])
val, test = train_test_split(temp, test_size=0.5, random_state=42, shuffle=True)
print(train.shape, test.shape, val.shape)

In [ ]:
val.head()

In [ ]:
train['select']= 'train'
test['select']='test'
val['select']="val"
train.head(5)

In [ ]:
dfdf = pd.read_csv('classes.csv', sep=';')
print(dfdf.head(5))
id_to_cla = {row["id"]: row["class"] for index, row in dfdf.iterrows()}
for kii,vii in id_to_cla.items():
    print(type(kii))

In [ ]:
df_train_val_test = pd.concat([train, val, test])

In [ ]:
df_train_val_test.head()

In [ ]:
def applydict_class(key):
    try:
        ret = id_to_cla[int(key)]
        #print(type(key))
    except Exception as err:
        #print(type(key))
        print(err)
        ret = 'noname found'
    return ret
df_train_val_test['class'] = df_train_val_test['labels'].apply(applydict_class)
df_train_val_test.head()
print(list(set(list(df_train_val_test['class']))))

In [ ]:
df_train_val_test.shape

In [ ]:
img_to_cl = {row["image"]: row["class"] for index, row in df_train_val_test.iterrows()}
img_to_fl = {row["image"]: row["select"] for index, row in df_train_val_test.iterrows()}

In [ ]:
all_class = list(set(df_train_val_test['class']))
print(len(all_class))

In [ ]:
for tvt in folders_to_create:
    for id in all_class:
        path = f"/Users/raphaelkronberg/PycharmProjects/deeploki/data/loki_subclasses/{tvt}/{id}"
        create_folder(path)

In [ ]:
import shutil

for root, dirs, files in os.walk("/Users/raphaelkronberg/Downloads/export_939_20220910_0950"):
    for name in files:
        if name.endswith('.png'):
            try:
                temp_class = img_to_cl[name]
                temp_folder = img_to_fl[name]
            except Exception as err:
                print('Not in Pool')
            temp_path = f"/Users/raphaelkronberg/PycharmProjects/deeploki/data/loki_subclasses/{temp_folder}/{temp_class}"
            src_file = os.path.join(root, name)
            #shutil.copy(src_file, temp_path)

In [ ]:
to_concat_df = []
for cls in all_class:
    print(len(classes))
    to_concat_df.append(data_table[data_table["labels"]==cls][:300])
data_table_small =pd.concat(to_concat_df)

In [ ]:
data_table_small

In [ ]:
from sklearn.model_selection import train_test_split
df = data_table_small
print(df.shape)
print(df.columns)
df['select'] = df['labels'].apply(is_in_df)
df = df[df['select']]
print(df.shape)
train, temp = train_test_split(df, test_size=0.2, train_size=0.8, random_state=42, shuffle=True, stratify=df['labels'])
val, test = train_test_split(temp, test_size=0.5, random_state=42, shuffle=True)
print(train.shape, test.shape, val.shape)

In [ ]:
train['select']= 'train'
test['select']='test'
val['select']="val"

In [ ]:
df_train_val_test_simple = pd.concat([train, val, test])
df_train_val_test_simple.shape


In [ ]:
img_to_cl2 = {row["image"]: row["labels"] for index, row in df_train_val_test_simple.iterrows()}
img_to_fl2 = {row["image"]: row["select"] for index, row in df_train_val_test_simple.iterrows()}
len(img_to_cl2)
len(img_to_fl2)

In [ ]:
import shutil

for root, dirs, files in os.walk("/Users/raphaelkronberg/Downloads/export_939_20220910_0950"):
    for name in files:
        if name.endswith('.png'):
            try:
                temp_class = img_to_cl2[name]
                temp_folder = img_to_fl2[name]
                print(temp_class, temp_folder)
            except Exception as err:
                pass
                #print('Not in Pool')
            temp_path = f"/Users/raphaelkronberg/PycharmProjects/deeploki/data/loki_small/{temp_folder}/{temp_class}"
            src_file = os.path.join(root, name)
            #print(temp_path)
            #shutil.copy(src_file, temp_path)
print(('end'))